<a href="https://colab.research.google.com/github/Abhi050/Deeplearning-in-Remote-Sensing/blob/main/major_project_with_slic_abhishek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install -q -U pip
!pip install -q cython
# Install pycocotools, the version by default in Colab
# has a bug fixed in https://github.com/cocodataset/cocoapi/pull/354
!pip install -q -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip install -q -U segmentation-models-pytorch
# !pip install -q -U albumentations

In [ ]:
import json
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.path as plt_path
  
from PIL import Image

from pathlib import Path
from pycocotools.coco import COCO

DATASET_PATH = Path('../input/maschusetbuilding')
img_root_path = DATASET_PATH / 'train/images'

In [ ]:
import albumentations as albu
from albumentations import (
    Compose, OneOf, Normalize, Resize, RandomResizedCrop, RandomCrop, HorizontalFlip, VerticalFlip, 
    RandomBrightness, RandomContrast, RandomBrightnessContrast, Rotate, ShiftScaleRotate, Cutout, 
    IAAAdditiveGaussianNoise, Transpose, Lambda
    )
from albumentations.pytorch import ToTensorV2

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
import segmentation_models_pytorch as smp

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

from torch.optim import Adam, SGD
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset

from torchvision.io import read_image
import torchvision.models as models

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
print(torch.__version__)
print(torchvision.__version__)
print(albu.__version__)

# Load Datasets

In [ ]:
anno_path = DATASET_PATH / 'masachuset-s.json'
with open(DATASET_PATH / 'masachuset-s.json', 'r') as f:
    annot_data = json.load(f)

In [ ]:
coco_anno = COCO(str(anno_path))
cat_ids = coco_anno.getCatIds()
cats = coco_anno.loadCats(cat_ids)
img_ids = coco_anno.getImgIds(catIds=cat_ids)

In [ ]:
def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


# https://stackoverflow.com/questions/50805634/how-to-create-mask-images-from-coco-dataset
def get_mask(coco, img_id, cat_ids):
    img_inf = coco.loadImgs(img_id)[0]
    img_shape = (img_inf['width'], img_inf['height'], len(cat_ids))
    ann_ids = coco_anno.getAnnIds(imgIds=img_id,
                                  catIds=cat_ids,
                                  iscrowd=None)
    anns = coco.loadAnns(ann_ids)
#     masks = np.zeros((img_inf['height'],img_inf['width'], len(cat_ids)))
#     for idx, cat_id in enumerate(cat_ids):
#         mask = np.zeros((img_inf['height'],img_inf['width']))
#         for ann in anns:
#             if cat_id == ann['category_id']:
# #                 mask = coco_anno.annToMask(ann)
#                 mask = np.maximum(mask, coco_anno.annToMask(ann))
#         masks[:, :, idx] = mask
    masks = np.zeros(img_shape)
    for idx, cat_id in enumerate(cat_ids):
        mask = np.zeros(img_shape[:2])
        for ann in anns:
            if cat_id == ann['category_id']:
                mask = np.maximum(mask, coco_anno.annToMask(ann))
        masks[:, :, idx] = mask
    return masks


def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()
    

seed_torch()

In [ ]:
def get_training_augmentation(width=320, height=320):
    train_transform = [

        albu.HorizontalFlip(p=0.5),

        albu.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=1, border_mode=0),

        albu.PadIfNeeded(min_height=height, min_width=width, always_apply=True, border_mode=0),
        albu.RandomCrop(height=height, width=width, always_apply=True),

        albu.IAAAdditiveGaussianNoise(p=0.2),
        albu.IAAPerspective(p=0.5),

        albu.OneOf(
            [
                albu.CLAHE(p=1),
                albu.RandomBrightness(p=1),
                albu.RandomGamma(p=1),
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.IAASharpen(p=1),
                albu.Blur(blur_limit=3, p=1),
                albu.MotionBlur(blur_limit=3, p=1),
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.RandomContrast(p=1),
                albu.HueSaturationValue(p=1),
            ],
            p=0.9,
        ),
    ]
    return albu.Compose(train_transform)


def get_validation_augmentation(width=320, height=320):
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
#         albu.PadIfNeeded(min_height=height, min_width=width)
        albu.Resize(width, height)
    ]
    return albu.Compose(test_transform)


def get_transform(resize, phase='train'):
    _transforms = []
    _mean = [0.485, 0.456, 0.406]
    _std = [0.229, 0.224, 0.225]
    if phase == 'train':
        _transforms.append(Resize(resize, resize))
        _transforms.append(HorizontalFlip(p=0.5))
        _transforms.append(VerticalFlip(p=0.5))
#         for t in [
#             RandomResizedCrop(resize, resize),
#             Transpose(p=0.5),
#             HorizontalFlip(p=0.5),
#             VerticalFlip(p=0.5),
#             ShiftScaleRotate(p=0.5)]:
#             _transforms.append(t)
        ,
    else:
        _transforms.append(Resize(resize, resize))
    # 
    _transforms.append(Normalize(
                mean=_mean,
                std=_std,
            ))
    _transforms.append(ToTensorV2())
    return Compose(_transforms)


def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')


def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform

    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose

    """

    _transform = [
        Lambda(image=preprocessing_fn),
        Lambda(image=to_tensor, mask=to_tensor)
    ]
    return Compose(_transform)


class SegDataset(Dataset):
    def __init__(self, root_dir, img_ids, cat_ids, coco_api, transforms=None, preprocessing=None):
        self.root_dir = root_dir
        self.img_ids = img_ids
        self.cat_ids = cat_ids
        self.coco_api = coco_api
        self.transforms = transforms
        self.preprocessing = preprocessing
    def _convertToLAB(self):
        try:
            import cv2
            self.labimg = cv2.cvtColor(img, cv2.COLOR_BGR2LAB).astype(np.float64)
        except ImportError:
            self.labimg = np.copy(self.img)
            for i in xrange(self.labimg.shape[0]):
                for j in xrange(self.labimg.shape[1]):
                    rgb = self.labimg[i, j]
                    self.labimg[i, j] = self._rgb2lab(tuple(reversed(rgb)))

    def _rgb2lab ( self, inputColor ) :

       num = 0
       RGB = [0, 0, 0]

       for value in inputColor :
           value = float(value) / 255

           if value > 0.04045 :
               value = ( ( value + 0.055 ) / 1.055 ) ** 2.4
           else :
               value = value / 12.92

           RGB[num] = value * 100
           num = num + 1

       XYZ = [0, 0, 0,]

       X = RGB [0] * 0.4124 + RGB [1] * 0.3576 + RGB [2] * 0.1805
       Y = RGB [0] * 0.2126 + RGB [1] * 0.7152 + RGB [2] * 0.0722
       Z = RGB [0] * 0.0193 + RGB [1] * 0.1192 + RGB [2] * 0.9505
       XYZ[ 0 ] = round( X, 4 )
       XYZ[ 1 ] = round( Y, 4 )
       XYZ[ 2 ] = round( Z, 4 )

       XYZ[ 0 ] = float( XYZ[ 0 ] ) / 95.047         # ref_X =  95.047   Observer= 2°, Illuminant= D65
       XYZ[ 1 ] = float( XYZ[ 1 ] ) / 100.0          # ref_Y = 100.000
       XYZ[ 2 ] = float( XYZ[ 2 ] ) / 108.883        # ref_Z = 108.883

       num = 0
       for value in XYZ :

           if value > 0.008856 :
               value = value ** ( 0.3333333333333333 )
           else :
               value = ( 7.787 * value ) + ( 16 / 116 )

           XYZ[num] = value
           num = num + 1

       Lab = [0, 0, 0]

       L = ( 116 * XYZ[ 1 ] ) - 16
       a = 500 * ( XYZ[ 0 ] - XYZ[ 1 ] )
       b = 200 * ( XYZ[ 1 ] - XYZ[ 2 ] )

       Lab [ 0 ] = round( L, 4 )
       Lab [ 1 ] = round( a, 4 )
       Lab [ 2 ] = round( b, 4 )

       return Lab

    def generateSuperPixels(self):
        self._initData()
        indnp = np.mgrid[0:self.height,0:self.width].swapaxes(0,2).swapaxes(0,1)
        for i in range(self.ITERATIONS):
            self.distances = self.FLT_MAX * np.ones(self.img.shape[:2])
            for j in xrange(self.centers.shape[0]):
                xlow, xhigh = int(self.centers[j][3] - self.step), int(self.centers[j][3] + self.step)
                ylow, yhigh = int(self.centers[j][4] - self.step), int(self.centers[j][4] + self.step)

                if xlow <= 0:
                    xlow = 0
                if xhigh > self.width:
                    xhigh = self.width
                if ylow <=0:
                    ylow = 0
                if yhigh > self.height:
                    yhigh = self.height

                cropimg = self.labimg[ylow : yhigh , xlow : xhigh]
                colordiff = cropimg - self.labimg[self.centers[j][4], self.centers[j][3]]
                colorDist = np.sqrt(np.sum(np.square(colordiff), axis=2))

                yy, xx = np.ogrid[ylow : yhigh, xlow : xhigh]
                pixdist = ((yy-self.centers[j][4])**2 + (xx-self.centers[j][3])**2)**0.5
                dist = ((colorDist/self.nc)**2 + (pixdist/self.ns)**2)**0.5

                distanceCrop = self.distances[ylow : yhigh, xlow : xhigh]
                idx = dist < distanceCrop
                distanceCrop[idx] = dist[idx]
                self.distances[ylow : yhigh, xlow : xhigh] = distanceCrop
                self.clusters[ylow : yhigh, xlow : xhigh][idx] = j

            for k in xrange(len(self.centers)):
                idx = (self.clusters == k)
                colornp = self.labimg[idx]
                distnp = indnp[idx]
                self.centers[k][0:3] = np.sum(colornp, axis=0)
                sumy, sumx = np.sum(distnp, axis=0)
                self.centers[k][3:] = sumx, sumy
                self.centers[k] /= np.sum(idx)

    def _initData(self):
        self.clusters = -1 * np.ones(self.img.shape[:2])
        self.distances = self.FLT_MAX * np.ones(self.img.shape[:2])

        centers = []
        for i in xrange(self.step, self.width - self.step/2, self.step):
            for j in xrange(self.step, self.height - self.step/2, self.step):
                
                nc = self._findLocalMinimum(center=(i, j))
                color = self.labimg[nc[1], nc[0]]
                center = [color[0], color[1], color[2], nc[0], nc[1]]
                centers.append(center)
        self.center_counts = np.zeros(len(centers))
        self.centers = np.array(centers)

    def createConnectivity(self):
        label = 0
        adjlabel = 0
        lims = self.width * self.height / self.centers.shape[0]
        dx4 = [-1, 0, 1, 0]
        dy4 = [0, -1, 0, 1]
        new_clusters = -1 * np.ones(self.img.shape[:2]).astype(np.int64)
        elements = []
        for i in xrange(self.width):
            for j in xrange(self.height):
                if new_clusters[j, i] == -1:
                    elements = []
                    elements.append((j, i))
                    for dx, dy in zip(dx4, dy4):
                        x = elements[0][1] + dx
                        y = elements[0][0] + dy
                        if (x>=0 and x < self.width and 
                            y>=0 and y < self.height and 
                            new_clusters[y, x] >=0):
                            adjlabel = new_clusters[y, x]
                count = 1
                c = 0
                while c < count:
                    for dx, dy in zip(dx4, dy4):
                        x = elements[c][1] + dx
                        y = elements[c][0] + dy

                        if (x>=0 and x<self.width and y>=0 and y<self.height):
                            if new_clusters[y, x] == -1 and self.clusters[j, i] == self.clusters[y, x]:
                                elements.append((y, x))
                                new_clusters[y, x] = label
                                count+=1
                    c+=1
                if (count <= lims >> 2):
                    for c in range(count):
                        new_clusters[elements[c]] = adjlabel
                    label-=1
                label+=1
        self.new_clusters = new_clusters
    
    def __getitem__(self, idx):
        img_id = self.img_ids[idx]
        img_inf = self.coco_api.loadImgs(img_id)[0]
        file_name = img_inf['file_name']
        file_path = self.root_dir / file_name
        img = Image.open(file_path).convert('RGB')
        mask = get_mask(self.coco_api, img_id, self.cat_ids)
        
        if self.transforms:
            augmented = self.transforms(image=np.array(img), mask=mask)
            img = augmented['image']
            mask = augmented['mask']

        if self.preprocessing:
            augmented = self.preprocessing(image=img, mask=mask)
            img = augmented['image']
            mask = augmented['mask']

        return img, mask
    
    def __len__(self):
        return len(self.img_ids)

In [ ]:
train_img_ids, valid_img_ids = train_test_split(img_ids, test_size=0.1, random_state=42)
valid_img_ids, test_img_ids = train_test_split(valid_img_ids, test_size=0.5, random_state=42)
print(len(train_img_ids), len(valid_img_ids), len(test_img_ids))

In [ ]:
sample_dataset = SegDataset(root_dir=img_root_path,
                           img_ids=train_img_ids,
                           cat_ids=cat_ids,
                           coco_api=coco_anno)
image, mask = sample_dataset[4] # get some sample
# visualize(
#     image=image, 
#     builing_mask=mask.squeeze(),
# )

In [ ]:
augmented_dataset = SegDataset(root_dir=img_root_path,
                               img_ids=train_img_ids,
                               cat_ids=cat_ids,
                               coco_api=coco_anno,
                               transforms=get_transform(resize=300, phase='train'))
for i in range(3):
    image, mask = augmented_dataset[4]
    visualize(image=image.numpy().transpose(1, 2, 0), mask=mask.squeeze())

# segmentation-models-pytorch

In [ ]:
EPOCHS = 100
BATCH_SIZE = 15

In [ ]:
ENCODER = 'efficientnet-b5'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = ['building']
ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multicalss segmentation

# create segmentation model with pretrained encoder
model = smp.Unet(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=len(CLASSES), 
    activation=ACTIVATION,
)

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

In [ ]:
_preprocessing = get_preprocessing(preprocessing_fn)

train_dataset = SegDataset(root_dir=img_root_path,
                           img_ids=train_img_ids,
                           cat_ids=cat_ids,
                           coco_api=coco_anno,
                           transforms=get_training_augmentation(288, 288),
                           preprocessing=_preprocessing)
valid_dataset = SegDataset(root_dir=img_root_path,
                           img_ids=valid_img_ids,
                           cat_ids=cat_ids,
                           coco_api=coco_anno,
                           transforms=get_validation_augmentation(288, 288),
                           preprocessing=_preprocessing)

In [ ]:
train_loader = DataLoader(train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          num_workers=4,
                          pin_memory=True,
                          drop_last=True)
valid_loader = DataLoader(valid_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          num_workers=4,
                          pin_memory=True,
                          drop_last=True)

In [ ]:
loss = smp.utils.losses.DiceLoss()
metrics = [
    smp.utils.metrics.IoU(threshold=0.6), smp.utils.metrics.Fscore(threshold=0.6), smp.utils.metrics.Precision(threshold=0.6) ,
    smp.utils.metrics.Recall(threshold=0.6) ,
    smp.utils.metrics.Accuracy(threshold=0.6)
]
optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=0.0001),
])


# Traininig

In [ ]:
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=device,
    verbose=True,
)

valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    device=device,
    verbose=True,
)

In [ ]:

max_score = 0


#train accurascy, train loss, val_accuracy, val_loss 
x_epoch_data = []
train_dice_loss = []
train_iou_score = []
valid_dice_loss = []
valid_iou_score = []
train_pre_loss = []
train_recall_score = []
valid_pre_loss = []
valid_recall_score = []
train_acc = []
valid_acc = []
train_fscore=[]
valid_fscore=[]

for i in range(EPOCHS):

    print(f'\nEpoch: {i + 1}')
    train_logs = train_epoch.run(train_loader)
    valid_logs = valid_epoch.run(valid_loader)

    x_epoch_data.append(i)
    train_dice_loss.append(train_logs['dice_loss'])
    train_iou_score.append(train_logs['iou_score'])
    valid_dice_loss.append(valid_logs['dice_loss'])
    valid_iou_score.append(valid_logs['iou_score'])
    train_pre_loss.append(train_logs['precision'])
    train_recall_score.append(train_logs['recall'])
    valid_pre_loss.append(valid_logs['precision'])
    valid_recall_score.append(valid_logs['recall'])
    train_acc.append(train_logs['accuracy'])
    valid_acc.append(valid_logs['accuracy'])
    train_fscore.append(train_logs['fscore'])
    valid_fscore.append(valid_logs['fscore'])

    # do something (save model, change lr, etc.)
    if EPOCHS%5==0:
        torch.save({
            'epoch': EPOCHS,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, './general_model_Unet_real.pth')
        
    if max_score < valid_logs['iou_score']:
        max_score1 = valid_logs['dice_loss']
        max_score2 =valid_logs['iou_score']
        max_score3 =valid_logs['precision']
        max_score4 =valid_logs['recall']
        max_score5 =valid_logs['fscore']
        max_score6 =valid_logs['accuracy']
        torch.save(model, './best_model_Unet_real.pth')
        print('Model saved!')

    if i == 25:
        optimizer.param_groups[0]['lr'] = 1e-5
        print('Decrease decoder learning rate to 1e-5!')
        
        
torch.save(model,'./entire_model_Unet_real.pth' )

In [ ]:
# max score1 means dice loss, max_score2 means iou score,
# max_score3 means precision , max_score4 means recall
# max_score5 means fscore and max_score 6 means accuracy
print(max_score1,max_score2,max_score3,max_score4,max_score5,max_score6)

In [ ]:
fig = plt.figure(figsize=(14, 5))

ax1 = fig.add_subplot(1, 2, 1)
line1, = ax1.plot(x_epoch_data,train_dice_loss,label='train') 
line2, = ax1.plot(x_epoch_data,valid_dice_loss,label='validation')
ax1.set_title("dice loss")
ax1.set_xlabel('epoch')
ax1.set_ylabel('dice_loss')
ax1.legend(loc='upper right')

ax2 = fig.add_subplot(1, 2, 2)
line1, = ax2.plot(x_epoch_data,train_iou_score,label='train')
line2, = ax2.plot(x_epoch_data,valid_iou_score,label='validation') 
ax2.set_title("iou score")
ax2.set_xlabel('epoch')
ax2.set_ylabel('iou_score')
ax2.legend(loc='upper left')



plt.show()

In [ ]:
fig = plt.figure(figsize=(14, 5))

ax3 = fig.add_subplot(1, 2, 1)
line1, = ax3.plot(x_epoch_data,train_pre_loss,label='train')
line2, = ax3.plot(x_epoch_data,valid_pre_loss,label='validation') 
ax3.set_title("Precision")
ax3.set_xlabel('epoch')
ax3.set_ylabel('Precision')
ax3.legend(loc='upper left')

ax4 = fig.add_subplot(1, 2, 2)
line1, = ax4.plot(x_epoch_data,train_recall_score,label='train')
line2, = ax4.plot(x_epoch_data,valid_recall_score,label='validation') 
ax4.set_title("Recall")
ax4.set_xlabel('epoch')
ax4.set_ylabel('Recall')
ax4.legend(loc='right')

plt.show()

In [ ]:
fig = plt.figure(figsize=(14, 5))

ax5 = fig.add_subplot(1, 2, 1)
line1, = ax5.plot(x_epoch_data,train_fscore,label='train')
line2, = ax5.plot(x_epoch_data,valid_fscore,label='validation') 
ax5.set_title("F score")
ax5.set_xlabel('epoch')
ax5.set_ylabel('F score')
ax5.legend(loc='upper left')

ax6 = fig.add_subplot(1, 2, 2)
line1, = ax6.plot(x_epoch_data,train_acc,label='train')
line2, = ax6.plot(x_epoch_data,valid_acc,label='validation') 
ax6.set_title("Accuracy")
ax6.set_xlabel('epoch')
ax6.set_ylabel('Accuracy')
ax6.legend(loc='upper right')

plt.show()

In [ ]:
best_model = torch.load('./best_model_Unet_real.pth')

In [ ]:
test_dataset = SegDataset(root_dir=img_root_path,
                           img_ids=test_img_ids,
                           cat_ids=cat_ids,
                           coco_api=coco_anno,
                           transforms=get_validation_augmentation(288, 288),
                           preprocessing=_preprocessing)
test_dataloader = DataLoader(test_dataset)

In [ ]:
test_epoch = smp.utils.train.ValidEpoch(
    model=best_model,
    loss=loss,
    metrics=metrics,
    device=device,
    verbose=True
)

logs = test_epoch.run(test_dataloader)

In [ ]:
for i in range(3):
    n = np.random.choice(len(test_dataset))
    image, gt_mask = test_dataset[n]

    gt_mask = gt_mask.squeeze()

    x_tensor = torch.from_numpy(image).to(device).unsqueeze(0)
    pr_mask = best_model.predict(x_tensor)
    pr_mask = (pr_mask.squeeze().cpu().numpy().round())

    visualize(
        image=image.transpose(1, 2, 0), 
        ground_truth_mask=gt_mask, 
        predicted_mask=pr_mask
    )